## Project: LEGO Inventory
### Name: Steph Roberts

![LegoSchema](lego_schema.PNG)

## Project Overview

In this project, we will load multiple tables from the [Rebrickable](https://rebrickable.com/) website to perform data analysis. Rebrickable is a versatile platform that helps users discover which LEGO sets can be assembled using the parts they already own. The site also identifies missing pieces required for building new sets and suggests which sets to purchase to acquire those parts. This project will involve data aggregation, grouping, function creation, and working with multi-indexing. The schema image referenced above originates from [Rebrickable's lego database](https://rebrickable.com/help/lego-database/).

## Data

The data has been sourced directly from the Rebrickable website and includes files corresponding to the tables depicted in the schema. Your initial task will be to explore the data and familiarize yourself with the schema to understand how the tables are interrelated. Although a data dictionary is not provided, you can comprehend the dataset by reviewing the files, examining the schema, and consulting their Download page and API documentation.

Let's start by importing some essential libraries.

In [23]:
# common imports
import pandas as pd
import numpy as np

# Do not change this option; This allows the CodeGrade auto grading to function correctly
pd.set_option('display.max_columns', None)

## Import Data

We begin by loading the LEGO datasets, naming each one according to the table names provided in the schema. As we proceed, we’ll consider which datasets are essential for our analysis, acknowledging that not all of them may be needed. A key aspect of any data science project is determining the relevance of the available data. By examining the schema, we can better understand the connections between these datasets.


In [24]:
# import Lego data sets here
# make sure that files are in the same folder as your notebook
inventories = pd.read_csv('inventories.csv')
inventory_sets = pd.read_csv('inventory_sets.csv')
sets = pd.read_csv('sets.csv')
themes = pd.read_csv('themes.csv')
inventory_minifigs = pd.read_csv('inventory_minifigs.csv')
minifigs = pd.read_csv('minifigs.csv')
inventory_parts = pd.read_csv('inventory_parts.csv')
part_categories = pd.read_csv('part_categories.csv')
parts = pd.read_csv('parts.csv')
colors = pd.read_csv('colors.csv')
part_relationships = pd.read_csv('part_relationships.csv')
elements = pd.read_csv('elements.csv')

## Exploration

In the exploration phase, we begin by examining each dataset with functions like `.head()`, `.info()`, and `.describe()` to deepen our understanding of the data. Through this review, we can also observe the relationships between the various tables.

In [25]:
# explore the data
print(f'Inventories\n', inventories.head())
print(f'Inventory sets\n',inventory_sets.head())
print(f'Sets\n', sets.head())
print(f'Themes\n', themes.head())
print(f'Inventory minifigs\n', inventory_minifigs.head())
print(f'Mini Figs\n', minifigs.head())
print(f'Inventory Parts\n', inventory_parts.head())
print(f'Part Categories\n', part_categories.head())
print(f'Parts\n', parts.head())
print(f'Colors\n', colors.head())
print(f'Part Relationship\n', part_relationships.head())
print(f'Elements\n', elements.head())

Inventories
    id  version set_num
0   1        1  7922-1
1   3        1  3931-1
2   4        1  6942-1
3  15        1  5158-1
4  16        1   903-1
Inventory sets
    inventory_id  set_num  quantity
0            35  75911-1         1
1            35  75912-1         1
2            39  75048-1         1
3            39  75053-1         1
4            50   4515-1         1
Sets
   set_num                        name  year  theme_id  num_parts
0   001-1                       Gears  1965         1         43
1  0011-2           Town Mini-Figures  1979        67         12
2  0011-3  Castle 2 for 1 Bonus Offer  1987       199          0
3  0012-1          Space Mini-Figures  1979       143         12
4  0013-1          Space Mini-Figures  1979       143         12
Themes
    id            name  parent_id
0   1         Technic        NaN
1   3     Competition        1.0
2   4  Expert Builder        1.0
3  16      RoboRiders        1.0
4  17  Speed Slammers        1.0
Inventory minifigs
  

In [26]:
minifigs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11827 entries, 0 to 11826
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   fig_num    11827 non-null  object
 1   name       11827 non-null  object
 2   num_parts  11827 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 277.3+ KB


In [27]:
sets.describe()

,year,theme_id,num_parts
count,18576.000000,18576.000000,18576.000000
mean,2006.200581,405.924365,160.666344
std,13.749798,197.809058,395.885329
min,1949.000000,1.000000,0.000000
25%,2000.000000,246.000000,6.000000
50%,2010.000000,473.000000,37.000000
75%,2017.000000,525.000000,147.000000
max,2022.000000,719.000000,11695.000000


Next, we create a function named `data_count` that accepts a dataset (such as `sets`) and returns the total number of rows as an integer.

In [28]:
# create function for finding number of rows
def data_count(df):
    return len(df)

Calling the `data_count` function on the `sets` DataFrame confirms that there are 18,576 sets represented in the data.

In [29]:
# verify function
data_count(sets)

18576

Next, we explore the data to determine the average number of parts per LEGO set, rounding the result to the nearest integer.

In [30]:
# Find average number in lego set
round(sets['num_parts'].mean())

161

We also calculate the median number of parts per set, ensuring the result is presented as an integer.

In [31]:
# Find median number of parts
round(sets['num_parts'].median())

37

The median number of parts is significantly lower than the average, indicating that while most LEGO sets have fewer parts, there are some sets with a very high number of parts that increase the average.

Let's identify the set with the largest number of parts to see just how many pieces the most complex LEGO set contains.

In [32]:
# Find largest number of parts in a set
sets['num_parts'].max()

11695

Using the result from our previous calculation, we can now select the row from the `sets` DataFrame that corresponds to the set with the largest number of parts.

In [33]:
# select row with largest number of parts
sets.loc[[sets['num_parts'].idxmax()]]

,set_num,name,year,theme_id,num_parts
3867,31203-1,World Map,2021,709,11695


We create a function named `select_set_row` that takes a `set_num` as a string and returns the corresponding row from the `sets` DataFrame.

In [34]:
# create a select row function based on the set id number
def select_set_row(set_num):
    return sets[sets['set_num'] == set_num]

In [35]:
# verify set num function
select_set_row('31203-1')

,set_num,name,year,theme_id,num_parts
3867,31203-1,World Map,2021,709,11695


Next, we develop a function named `select_set_numparts` that accepts a specific number of parts as an integer. This function will select all rows in the `sets` DataFrame with that number of parts and return a DataFrame sorted by `year` and then by `set_num`, without resetting the index.

In [36]:
# create function to select rows with certain number of parts
def select_set_numparts(num_parts):
    return sets[sets['num_parts'] == num_parts ].sort_values(by =['year','set_num'])

We can use the `select_set_numparts()` function to identify all sets with `0` parts. This query should return 3,146 rows.

In [37]:
# verify select set numparts function
select_set_numparts(0)

,set_num,name,year,theme_id,num_parts
11061,700.1-2,Extra-Large Gift Set (Mursten),1953,366,0
17563,b55dk-01,Lego Mursten - System i Leg Byggebog,1955,497,0
17564,b56de-01,System im Spiel,1956,497,0
11072,700.2-2,Gift Package,1957,366,0
11078,700.3A-2,Gift Package,1957,372,0
...,...,...,...,...,...
15619,854195-1,Ernie Key Chain,2022,503,0
15620,854197-1,Demogorgon Key Chain,2022,503,0
17170,9780744054583-1,DC Comics Super Heroes: Character Encyclopedia...,2022,497,0
17193,9780794447557-1,City: Color The City,2022,497,0


Let's identify the oldest set in the dataset by finding the earliest year listed.
Similarly, we'll determine the newest set by finding the most recent year in the dataset.

In [38]:
# find the oldest set year
oldest_set = sets['year'].min()
oldest_set

1949

In [39]:
# find the newest set year
newest_set = sets['year'].max()
newest_set

2022

We'll create a function named `select_set_year` that takes a year as an integer input and returns the rows from the `sets` DataFrame that correspond to that year. The returned DataFrame will be sorted by `set_num` without resetting the index.

In [40]:
# create a function for finding sets made in a particular year
def select_set_year(year):
    return sets[sets['year'] == year].sort_values(['set_num'])

In [41]:
# verify set year function
select_set_year(2022)

,set_num,name,year,theme_id,num_parts
336,10297-1,Boutique Hotel,2022,155,3068
600,10781-1,Spider-Man’s Techno Trike,2022,706,59
601,10782-1,Hulk vs. Rhino Truck Showdown,2022,706,110
602,10783-1,Spider-Man at Doc Ock’s Lab,2022,706,131
603,10784-1,Spider-Man Webquarters Hangout,2022,706,155
...,...,...,...,...,...
17170,9780744054583-1,DC Comics Super Heroes: Character Encyclopedia...,2022,497,0
17193,9780794447557-1,City: Color The City,2022,497,0
17200,9780794448172-1,DC Comics Super Heroes: Ready For Action,2022,497,0
17207,9780794449247-1,DC Comics Super Heroes: Gotham City’s New Defe...,2022,497,7


We will create a function named `theme_by_year` that takes a year as an integer and returns a DataFrame showing the `id` and `name_themes` of themes that appeared in sets during that year. The DataFrame will be sorted by `id`, with the index reset to run from `0` to `n-1`. Each theme will be listed only once, ensuring that duplicate themes are identified by `id` rather than by name, as some themes may share the same name but have different ids.

In [42]:
# first we can merge the sets and themes dfs
themed_sets = sets.merge(themes.rename(columns={'name': 'name_themes'}), 
                         left_on='theme_id', right_on='id').drop(columns='theme_id')
themed_sets

,set_num,name,year,num_parts,id,name_themes,parent_id
0,001-1,Gears,1965,43,1,Technic,NaN
1,0011-2,Town Mini-Figures,1979,12,67,Classic Town,50.0
2,0011-3,Castle 2 for 1 Bonus Offer,1987,0,199,Lion Knights,186.0
3,0012-1,Space Mini-Figures,1979,12,143,Supplemental,126.0
4,0013-1,Space Mini-Figures,1979,12,143,Supplemental,126.0
...,...,...,...,...,...,...,...
18571,XWING-1,Mini X-Wing Fighter,2019,60,158,Star Wars,NaN
18572,XWING-2,X-Wing Trench Run,2019,52,158,Star Wars,NaN
18573,YODACHRON-1,Yoda Chronicles Promotional Set,2013,413,158,Star Wars,NaN
18574,YTERRIER-1,Yorkshire Terrier,2018,0,598,Promotional,NaN


In [43]:
# create function that finds theme ids and names from year input
def theme_by_year(year):
    return themed_sets[themed_sets['year'] == year][['id','name_themes']]\
        .sort_values(['id']).drop_duplicates(['id']).reset_index(drop=True)

In [44]:
# verify theme by year function
theme_by_year(1960)

,id,name_themes
0,371,Supplemental
1,497,Books
2,513,Classic


In [45]:
# call theme by year function on my birth year
theme_by_year(1990)

,id,name_themes
0,1,Technic
1,21,Universal Building Set
2,67,Classic Town
3,128,Blacktron I
4,132,Futuron
5,136,M:Tron
6,188,Black Knights
7,190,Crusaders
8,194,Forestmen
9,229,Easter


In [46]:
themed_sets.head()

,set_num,name,year,num_parts,id,name_themes,parent_id
0,001-1,Gears,1965,43,1,Technic,NaN
1,0011-2,Town Mini-Figures,1979,12,67,Classic Town,50.0
2,0011-3,Castle 2 for 1 Bonus Offer,1987,0,199,Lion Knights,186.0
3,0012-1,Space Mini-Figures,1979,12,143,Supplemental,126.0
4,0013-1,Space Mini-Figures,1979,12,143,Supplemental,126.0


Next, we'll develop a function named `theme_by_name` that takes a theme name as input and outputs all sets associated with that theme, ordered by `set_num`. The resulting DataFrame will include the columns `set_num`, `name_sets`, `year`, `num_parts`, `theme_id`, and `name_themes`, in that specific order. The index will be reset to run from `0` to `n-1`.

In [47]:
# create a function to find the sets with certain themes
def theme_by_name(theme):
    return themed_sets.rename(columns={'name':'name_sets','id':'theme_id'})[themed_sets['name_themes'] == theme]\
        .drop(columns='parent_id').sort_values(['set_num']).reset_index(drop=True)
    

We can now call the `theme_by_name()` function with the theme name `Gear`. This should return 1,904 sets that match the specified theme name.

In [48]:
# verify theme by name function
theme_by_name('Gear')

,set_num,name_sets,year,num_parts,theme_id,name_themes
0,11138-1,Bionicle Barraki Wallet,2008,0,501,Gear
1,119645111436-1,Classic Space Astronaut Plush (Red),2021,0,501,Gear
2,119645111504-1,Classic Space Astronaut Plush (Blue),2021,0,501,Gear
3,119645111740-1,Classic Space Astronaut Plush (White),2021,0,501,Gear
4,119645111818-1,Classic Space Astronaut Plush (Pink),2021,0,501,Gear
...,...,...,...,...,...,...
1899,WATCH-3,Watch System Voyager (UFO Space),1997,29,501,Gear
1900,WIIGAME-1,Bionicle Heroes - Wii,2007,0,501,Gear
1901,WIIGAME-2,Indiana Jones: The Original Adventures Video G...,2008,0,501,Gear
1902,WOODENELEPHANT-1,Original Wooden Elephant Pull-Along Toy and Print,2021,0,501,Gear


Let's create a function named `theme_by_setnum` that takes a `set_num` as a string input and returns the corresponding theme name as a string.

In [49]:
# create a function that finds the theme name for a set number id
def theme_by_setnum(set_num):
    return themed_sets[themed_sets['set_num'] == set_num]['name_themes'].item()


To check our function , I visited the [Sets page](https://rebrickable.com/sets/) from the Rebrickable website and and found a set with the number '31203-1'.  Calling the `theme_by_setnum()` function on that set correctly identifies the theme name of `LOGE Art`. 

In [50]:
# verify the function
theme_by_setnum('31203-1')

'LEGO Art'

**Millennium Falcon Analysis:**

We begin by identifying all the sets named 'Millennium Falcon'. From this list, we determine which set contains the largest number of parts.


In [51]:
# find the Millennium Falcon set with the largest number of parts
MF = themed_sets[themed_sets['name'] == 'Millennium Falcon'].nlargest(1, 'num_parts')
MF

,set_num,name,year,num_parts,id,name_themes,parent_id
12884,75192-1,Millennium Falcon,2017,7541,171,Ultimate Collector Series,158.0


Next, we determine how many minifigs are included in the set identified from the 'Millennium Falcon' analysis.

Although there can be multiple versions of a set, as indicated in the `inventories` table, this detail won't affect our analysis in this case, but we'll consider it in future steps.

In [52]:
inventories.head(2)

,id,version,set_num
0,1,1,7922-1
1,3,1,3931-1


In [53]:
merged_sets = sets.merge(inventories, on='set_num') \
                  .merge(inventory_minifigs, left_on='id', right_on='inventory_id') \
                  .merge(minifigs, on='fig_num',suffixes=('_sets', '_mini'))\
                    .rename(columns={'name_mini': 'fig_name', 'num_parts_mini': 'fig_num_parts'})\
                    .drop(columns='id')

merged_sets.head()


,set_num,name_sets,year,theme_id,num_parts_sets,version,inventory_id,fig_num,quantity,fig_name,fig_num_parts
0,0011-2,Town Mini-Figures,1979,67,12,1,5087,fig-000225,1,"Policeman, Black Suit with Pocket and Badge, W...",4
1,0011-2,Town Mini-Figures,1979,67,12,1,5087,fig-000834,1,"Man, Red Torso, Blue Legs, Red Hard Hat",4
2,0011-2,Town Mini-Figures,1979,67,12,1,5087,fig-004401,1,"Woman - Blue Torso, Red Legs",4
3,0012-1,Space Mini-Figures,1979,143,12,1,1414,fig-000020,2,"Classic Spaceman, Red with Airtanks (3842a Hel...",5
4,0013-1,Space Mini-Figures,1979,143,12,1,4609,fig-001127,2,"Classic Spaceman, White with Airtanks (3842a H...",5


In [54]:
# find number of minifigs in the Millennium Falcon set with the largest number of parts (set_num = 75192-1)
MF_minifigs_num = merged_sets[merged_sets['set_num'] == '75192-1'].shape[0]
MF_minifigs_num


8

We'll create a function named `get_minifigs` that accepts a set number (as a string) and a version number (as an integer). This function will output all minifigs associated with that specific set/version combination. The resulting DataFrame will be ordered by `fig_num` and include the columns `set_num`, `fig_num`, `fig_name`, and `fig_num_parts`, with the index reset to run from 0 to n-1.

In [55]:
# create function to find all minifigs for a version/set combo
def get_minifigs(set_num, version): 
    return merged_sets[(merged_sets['set_num']==set_num)&(merged_sets['version']==version)]\
        [['set_num','fig_num','fig_name','fig_num_parts']].sort_values(['fig_num']).reset_index(drop=True)


We can now use the `get_minifigs` function to retrieve detailed information about the minifigs included in the set identified during the 'Millennium Falcon' analysis.

In [56]:
# find minifigs in the Millennium Falcon set
MF_minifigs = get_minifigs('75192-1',1)
MF_minifigs

,set_num,fig_num,fig_name,fig_num_parts
0,75192-1,fig-001714,"Chewbacca, Dark Brown",3
1,75192-1,fig-001810,"Finn, Black Undershirt",4
2,75192-1,fig-002057,"Rey, Dark Tan Robe",4
3,75192-1,fig-002514,"C-3PO, Pearl Gold, Colorful Wires, Printed Legs",3
4,75192-1,fig-002544,BB-8,2
5,75192-1,fig-002549,"Princess Leia, Hoth Outfit, White Jacket, Prin...",4
6,75192-1,fig-002550,"Han Solo, Dark Blue Jacket, Dark Brown Legs, P...",4
7,75192-1,fig-002551,"Han Solo, Old, Angry",4


To ensure the `get_minifigs()` function works correctly, we'll test it with both version 1 and version 2 of set 10217-1, Harry Potter's Diagon Alley. The function should return a DataFrame with 11 minifigs for each version.

In [57]:
# verify the get minifigs function
HP1 = len(get_minifigs('10217-1',1))
HP2 = len(get_minifigs('10217-1',2))
print(HP1,HP2)

11 11


In [58]:
merged_sets.head(2)

,set_num,name_sets,year,theme_id,num_parts_sets,version,inventory_id,fig_num,quantity,fig_name,fig_num_parts
0,0011-2,Town Mini-Figures,1979,67,12,1,5087,fig-000225,1,"Policeman, Black Suit with Pocket and Badge, W...",4
1,0011-2,Town Mini-Figures,1979,67,12,1,5087,fig-000834,1,"Man, Red Torso, Blue Legs, Red Hard Hat",4


In [59]:
themes.head(2)

,id,name,parent_id
0,1,Technic,NaN
1,3,Competition,1.0


We'll create a function named `minifigs_from_themes` that takes a theme id (as an integer) as input and outputs all the minifigs found in sets associated with that theme. The resulting DataFrame will be ordered by `fig_num` and include the columns `fig_num` and `fig_name`. The index will be reset to run from 0 to n-1, and any exact duplicates will be removed to ensure only unique minifigs are included.

In [60]:
# merge themes into merged_sets
merged_sets_theme = merged_sets.merge(themes, left_on='theme_id',right_on='id', suffixes=('_set','_theme'))\
    .rename(columns={'name':'theme_name'}).drop(columns='id')
merged_sets_theme.head(2)

,set_num,name_sets,year,theme_id,num_parts_sets,version,inventory_id,fig_num,quantity,fig_name,fig_num_parts,theme_name,parent_id
0,0011-2,Town Mini-Figures,1979,67,12,1,5087,fig-000225,1,"Policeman, Black Suit with Pocket and Badge, W...",4,Classic Town,50.0
1,0011-2,Town Mini-Figures,1979,67,12,1,5087,fig-000834,1,"Man, Red Torso, Blue Legs, Red Hard Hat",4,Classic Town,50.0


In [61]:
# create function to find all minifigs in sets with a specifc theme
def minifigs_from_themes(theme_id):
    return merged_sets_theme[merged_sets_theme['theme_id']==theme_id][['fig_num','fig_name']]\
        .sort_values(['fig_num']).drop_duplicates().reset_index(drop=True)

We can now call the `minifigs_from_themes()` function using the theme number associated with the set from the 'Millennium Falcon' analysis. This should return a total of 149 unique minifigs for that theme.

In [62]:
# verify minifigs theme function
minifigs_from_themes(171)

,fig_num,fig_name
0,fig-000003,"Assassin Droid, White"
1,fig-000124,"Chewbacca, Reddish Brown"
2,fig-000144,"Luke Skywalker, White Robe, White Legs, Cheek ..."
3,fig-000303,"Clone Trooper, Phase I Armor, Brown Eyes"
4,fig-000516,"Darth Vader, Light Bluish Gray Skin, Scarred R..."
...,...,...
144,fig-011942,"AT-AT Driver - Light Bluish Grey Helmet, Sand ..."
145,fig-011944,"Snowtrooper, White Hips, Reddish Brown Head, F..."
146,fig-011945,"Snowtrooper, White Hips, Light Flesh Head, Female"
147,fig-011946,"Snowtrooper, White Hips, Light Flesh Head, Off..."


Create a function named `sets_from_minifig` that takes a minifig number (as a string) as input and returns a list of sets in which that minifig has appeared. The function will return a DataFrame sorted by `set_num` and then by `version`, including the columns `set_num`, `version`, `name_sets`, `fig_num`, and `fig_name`. The index will be reset to run from 0 to n-1.

In [63]:
# create function to return a list of sets a minifig is included in
def sets_from_minifig(fig_num):
    return merged_sets_theme[merged_sets_theme['fig_num']==fig_num][['set_num','version','name_sets','fig_num','fig_name']]\
        .sort_values(['set_num','version']).reset_index(drop=True)

To verify the `sets_from_minifig()` function, we'll call it using the minifig number `fig-001039` for Andrea from the Lego Friends series. The function should return a DataFrame that includes the following sets:

- Andrea on the Beach
- Birthday Party
- City Park Cafe

In [64]:
# verify sets from minifigs function
sets_from_minifig('fig-001039')

,set_num,version,name_sets,fig_num,fig_name
0,30100-1,1,Andrea on the Beach,fig-001039,"Andrea - Bright Light Orange Top, Light Aqua S..."
1,30107-1,1,Birthday Party,fig-001039,"Andrea - Bright Light Orange Top, Light Aqua S..."
2,3061-1,1,City Park Café,fig-001039,"Andrea - Bright Light Orange Top, Light Aqua S..."


Let's create a function named `set_parts` that takes a set number (as a string) and a version number (as an integer) as inputs. This function will return a DataFrame listing each part in the specified set. The DataFrame will include the following columns: `set_num`, `version`, `name_sets`, `year`, `name_themes`, `part_num`, `name_part`, `name_color`, `quantity`, and `is_spare`. The data will be sorted by `part_num` and then by `name_color`, with the index reset to run from 0 to n-1.

To ensure we stay within the memory limits, we will only merge the necessary datasets: `themes`, `sets`, `inventories`, `inventory_parts`, `parts`, and `colors`, avoiding the inclusion of unrelated data that could increase memory usage.

In [65]:
merged_parts = sets.merge(themes, left_on='theme_id', right_on='id', suffixes=('_sets','_themes'))\
                  .merge(inventories, on='set_num', suffixes=('_themes','_inv'))\
                  .merge(inventory_parts, left_on='id_inv', right_on='inventory_id')\
                  .merge(parts, on='part_num')\
                  .merge(colors, left_on='color_id', right_on='id', suffixes=('_part','_color'))\
                  [['set_num','version','name_sets','year','name_themes','part_num','name_part','name_color','quantity','is_spare']]\
                  .reset_index(drop=True)
merged_parts

,set_num,version,name_sets,year,name_themes,part_num,name_part,name_color,quantity,is_spare
0,001-1,1,Gears,1965,Technic,132a,Tyre Smooth Old Style - Small,Light Gray,4,f
1,001-1,1,Gears,1965,Technic,3020,Plate 2 x 4,White,4,f
2,001-1,1,Gears,1965,Technic,3062c,"Brick Round 1 x 1 Solid Stud, No Bottom Groove",White,1,f
3,001-1,1,Gears,1965,Technic,3404bc01,"Turntable 4 x 4 - Old Type Complete, Perfectly...",White,4,f
4,001-1,1,Gears,1965,Technic,36,Tyre Smooth Old Style - Large,Light Gray,4,f
...,...,...,...,...,...,...,...,...,...,...
935303,YODACHRON-1,1,Yoda Chronicles Promotional Set,2013,Star Wars,6541,Technic Brick 1 x 1 with Hole,Light Bluish Gray,12,f
935304,YODACHRON-1,1,Yoda Chronicles Promotional Set,2013,Star Wars,6636,Tile 1 x 6 with Groove,White,4,f
935305,YODACHRON-1,1,Yoda Chronicles Promotional Set,2013,Star Wars,87079,Tile 2 x 4 with Groove,Dark Bluish Gray,3,f
935306,YODACHRON-1,1,Yoda Chronicles Promotional Set,2013,Star Wars,87083,Technic Axle 4 with Stop,Dark Bluish Gray,1,f


In [66]:
# create a function that returns each part in the set from a set number/version combo 
def set_parts(set_num, version):
    return merged_parts[(merged_parts['set_num']==set_num) & (merged_parts['version']==version)]\
        .sort_values(['part_num','name_color']).reset_index(drop=True)  

Using the `set_parts()` function, we'll output a list of parts for the set discussed in the 'Millennium Falcon' analysis. The resulting DataFrame should contain a total of 730 rows, which includes 676 parts and 54 spare parts. You can verify these results on the respective page of the Rebrickable website.

In [67]:
# verify set parts function
set_parts('75192-1',1)

,set_num,version,name_sets,year,name_themes,part_num,name_part,name_color,quantity,is_spare
0,75192-1,1,Millennium Falcon,2017,Ultimate Collector Series,10197,Technic Pin Connector Hub with 2 Perpendicular...,Dark Bluish Gray,3,f
1,75192-1,1,Millennium Falcon,2017,Ultimate Collector Series,10201,Bracket 1 x 2 - 1 x 4 [Rounded Corners],Light Bluish Gray,12,f
2,75192-1,1,Millennium Falcon,2017,Ultimate Collector Series,10247,Plate Special 2 x 2 with 1 Pin Hole [Complete ...,Dark Bluish Gray,5,f
3,75192-1,1,Millennium Falcon,2017,Ultimate Collector Series,10247,Plate Special 2 x 2 with 1 Pin Hole [Complete ...,White,22,f
4,75192-1,1,Millennium Falcon,2017,Ultimate Collector Series,10314,"Brick Curved 1 x 4 x 1 1/3 No Studs, Curved To...",Tan,3,f
...,...,...,...,...,...,...,...,...,...,...
725,75192-1,1,Millennium Falcon,2017,Ultimate Collector Series,99563,Tile Special 1 x 2 with Sloped Walls AKA Money...,Light Bluish Gray,70,f
726,75192-1,1,Millennium Falcon,2017,Ultimate Collector Series,99780,Bracket 1 x 2 - 1 x 2 Inverted,Black,6,f
727,75192-1,1,Millennium Falcon,2017,Ultimate Collector Series,99780,Bracket 1 x 2 - 1 x 2 Inverted,Dark Bluish Gray,25,f
728,75192-1,1,Millennium Falcon,2017,Ultimate Collector Series,99781,Bracket 1 x 2 - 1 x 2,Light Bluish Gray,2,f
